In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Definir las transformaciones para los datos de prueba
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Descargar y cargar el dataset de prueba
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

# Nombres de las clases
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Función para mostrar una imagen
def imshow(img):
    img = img / 2 + 0.5  # Desnormalizar
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Obtener algunas imágenes de prueba
dataiter = iter(testloader)
images, labels = next(dataiter)

# Mostrar imágenes
imshow(torchvision.utils.make_grid(images))
# Imprimir etiquetas
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))



Files already downloaded and verified


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms


transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Descargar y cargar el dataset de entrenamiento y prueba
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

import torch.nn.functional as F

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):  # número de épocas

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Obtener los inputs; los datos son una lista de [inputs, labels]
        inputs, labels = data

        # Inicializar gradientes a cero
        optimizer.zero_grad()

        # Hacer forward, backward y optimizar
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Imprimir estadísticas
        running_loss += loss.item()
        if i % 2000 == 1999:    # imprimir cada 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')

100%|███████████████████████████████████████████████████████████████| 170498071/170498071 [02:33<00:00, 1112305.95it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified
[1,  2000] loss: 2.153
[1,  4000] loss: 1.792
[1,  6000] loss: 1.659
[1,  8000] loss: 1.604
[1, 10000] loss: 1.512
[1, 12000] loss: 1.484
[2,  2000] loss: 1.383
[2,  4000] loss: 1.373
[2,  6000] loss: 1.329
[2,  8000] loss: 1.318
[2, 10000] loss: 1.282
[2, 12000] loss: 1.245
Finished Training
Accuracy of the network on the 10000 test images: 54.86 %


In [ ]:
# Guardar el modelo
torch.save(net.state_dict(), 'cifar10_net.pth')

# Cargar el modelo
net = Net()
net.load_state_dict(torch.load('cifar10_net.pth'))